# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,guerrero negro,27.9769,-114.0611,26.09,24,88,1.82,MX,1702938288
1,1,port-aux-francais,-49.3500,70.2167,7.32,95,99,17.81,TF,1702938288
2,2,newman,37.3138,-121.0208,16.83,69,100,4.80,US,1702938288
3,3,anadyr,64.7500,177.4833,-18.72,84,100,5.00,RU,1702938288
4,4,ilulissat,69.2167,-51.1000,-8.99,62,75,11.32,GL,1702938289


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_city = city_data_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", \
                                        frame_width=700, frame_height=500, \
                                        size="Humidity", scale=0.5, color="City")
# Display the map
map_plot_city

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_ideal = ((city_data_df["Max Temp"] >= 17) & (city_data_df["Max Temp"] <= 30))
wind_ideal = (city_data_df["Wind Speed"] < 4)
# Drop any rows with null values
city_data = city_data_df.loc[(temp_ideal & wind_ideal), :]
city_ideal = city_data.dropna()

# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,guerrero negro,27.9769,-114.0611,26.09,24,88,1.82,MX,1702938288
15,15,beitbridge,-22.2167,30.0000,27.65,67,100,3.55,ZW,1702938291
19,19,holualoa,19.6228,-155.9522,24.91,77,75,3.60,US,1702938044
24,24,maxixe,-23.8597,35.3472,24.58,91,41,2.42,MZ,1702938293
31,31,olinda,-8.0089,-34.8553,27.85,74,40,3.60,BR,1702938294
...,...,...,...,...,...,...,...,...,...,...
540,540,felipe carrillo puerto,19.5833,-88.0500,25.30,50,0,2.02,MX,1702938375
546,546,benguela,-12.5763,13.4055,26.19,86,100,1.34,AO,1702938392
552,552,mabaruma,8.2000,-59.7833,25.01,89,67,1.95,GY,1702938394
559,559,adiake,5.2863,-3.3040,26.75,83,27,3.30,CI,1702938395


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_ideal[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\cjsua\AppData\Local\Temp\ipykernel_15992\4113061349.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,guerrero negro,MX,27.9769,-114.0611,24,
15,beitbridge,ZW,-22.2167,30.0000,67,
19,holualoa,US,19.6228,-155.9522,77,
24,maxixe,MZ,-23.8597,35.3472,91,
31,olinda,BR,-8.0089,-34.8553,74,
...,...,...,...,...,...,...
540,felipe carrillo puerto,MX,19.5833,-88.0500,50,
546,benguela,AO,-12.5763,13.4055,86,
552,mabaruma,GY,8.2000,-59.7833,89,
559,adiake,CI,5.2863,-3.3040,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accommodation.hotel"]
params = {"categories": categories,
    "apiKey": geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
guerrero negro - nearest hotel: Plaza sal paraiso
beitbridge - nearest hotel: Beitbridge Inn & Casino
holualoa - nearest hotel: Kona Hotel
maxixe - nearest hotel: Hotel Herry
olinda - nearest hotel: Fidji Motel
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
uturoa - nearest hotel: Hawaiki Nui hotel
san antonio de pale - nearest hotel: No hotel found
guantanamo - nearest hotel: Hotel Martí
port elizabeth - nearest hotel: Waterford Hotel
laikit, laikit ii (dimembe) - nearest hotel: Chandra Lestari Hotel Cottage & Restoran
harper - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
sur - nearest hotel: Sur Hotel
chonchi - nearest hotel: Hotel Huildín
sinabang - nearest hotel: No hotel found
malango - nearest hotel: Madarana
parintins - nearest hotel: No hotel found
sao felix do xingu - nearest hotel: Hotel Terraço
kasane - nearest hotel: Kasane Travel Lodge
yashikira - nearest hotel: No hotel found
palmas - nearest hotel: Hotel 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,guerrero negro,MX,27.9769,-114.0611,24,Plaza sal paraiso
15,beitbridge,ZW,-22.2167,30.0000,67,Beitbridge Inn & Casino
19,holualoa,US,19.6228,-155.9522,77,Kona Hotel
24,maxixe,MZ,-23.8597,35.3472,91,Hotel Herry
31,olinda,BR,-8.0089,-34.8553,74,Fidji Motel
...,...,...,...,...,...,...
540,felipe carrillo puerto,MX,19.5833,-88.0500,50,Hotel María José
546,benguela,AO,-12.5763,13.4055,86,Hotel Moibela
552,mabaruma,GY,8.2000,-59.7833,89,Monty Brooms Hotel
559,adiake,CI,5.2863,-3.3040,83,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", \
                                        frame_width=700, frame_height=500, \
                                        size="Humidity", scale=0.5, color="City", \
                                        hover_cols = ["Hotel Name","Country"])

# Display the map
map_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)